In [ ]:
# === GOOGLE DRIVE MOUNT ===
# Mount Google Drive to access datasets, model checkpoints, and results
from google.colab import drive
drive.mount('/content/drive')
# store data here for notebook access

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# === IMPORTS ===
# Import necessary libraries for model loading, dataset handling, and visualization
from torch.utils.data import Dataset
# dataset class in pytorch which is the data that will get fed into the NN

In [ ]:
# === IMPORTS ===
# Import necessary libraries for model loading, dataset handling, and visualization
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [ ]:
# === IMPORTS ===
# Import necessary libraries for model loading, dataset handling, and visualization
import shutil
import random

In [ ]:
# === IMPORTS ===
# Import necessary libraries for model loading, dataset handling, and visualization
from torch.utils.data import Dataset as BaseDataset

In [ ]:
# === IMPORTS ===
# Import necessary libraries for model loading, dataset handling, and visualization
import albumentations as albu
# https://albumentations.ai/docs/api_reference/augmentations/geometric/resize/

In [ ]:
# === DATA AUGMENTATION AND PREPROCESSING ===
# Define augmentation and preprocessing pipelines for validation and inference


def get_validation_augmentation():
    """Add paddings to make image shape divisible by 32"""
    test_transform = [
        albu.PadIfNeeded(544, 960)
    ]
    return albu.Compose(test_transform)

def get_preprocessing(preprocessing_fn):
    _transform = [
        albu.Lambda(image=preprocessing_fn),
        albu.Lambda(image=to_tensor, mask=to_tensor),
    ]
    return albu.Compose(_transform)

# Assuming the following functions and constants are defined correctly elsewhere in the code
# ENCODER, ENCODER_WEIGHTS, ACTIVATION, CLASSES, etc.

In [ ]:
# === IMPORTS ===
# Import necessary libraries for model loading, dataset handling, and visualization
import re
import torch.nn as nn


class BaseObject(nn.Module):
    def __init__(self, name=None):
        super().__init__()
        self._name = name

    @property
    def __name__(self):
        if self._name is None:
            name = self.__class__.__name__
            s1 = re.sub("(.)([A-Z][a-z]+)", r"\1_\2", name)
            return re.sub("([a-z0-9])([A-Z])", r"\1_\2", s1).lower()
        else:
            return self._name


class Metric(BaseObject):
    pass


class Loss(BaseObject):
    def __add__(self, other):
        if isinstance(other, Loss):
            return SumOfLosses(self, other)
        else:
            raise ValueError("Loss should be inherited from `Loss` class")

    def __radd__(self, other):
        return self.__add__(other)

    def __mul__(self, value):
        if isinstance(value, (int, float)):
            return MultipliedLoss(self, value)
        else:
            raise ValueError("Loss should be inherited from `BaseLoss` class")

    def __rmul__(self, other):
        return self.__mul__(other)


class SumOfLosses(Loss):
    def __init__(self, l1, l2):
        name = "{} + {}".format(l1.__name__, l2.__name__)
        super().__init__(name=name)
        self.l1 = l1
        self.l2 = l2

    def __call__(self, *inputs):
        return self.l1.forward(*inputs) + self.l2.forward(*inputs)


class MultipliedLoss(Loss):
    def __init__(self, loss, multiplier):

        # resolve name
        if len(loss.__name__.split("+")) > 1:
            name = "{} * ({})".format(multiplier, loss.__name__)
        else:
            name = "{} * {}".format(multiplier, loss.__name__)
        super().__init__(name=name)
        self.loss = loss
        self.multiplier = multiplier

    def __call__(self, *inputs):
        return self.multiplier * self.loss.forward(*inputs)
class Activation(nn.Module):

    def __init__(self, name, **params):

        super().__init__()

        if name is None or name == 'identity':
            self.activation = nn.Identity(**params)
        elif name == 'sigmoid':
            self.activation = nn.Sigmoid()
        elif name == 'softmax2d':
            self.activation = nn.Softmax(dim=1, **params)
        elif name == 'softmax':
            self.activation = nn.Softmax(**params)
        elif name == 'logsoftmax':
            self.activation = nn.LogSoftmax(**params)
        elif name == 'argmax':
            self.activation = ArgMax(**params)
        elif name == 'argmax2d':
            self.activation = ArgMax(dim=1, **params)
        elif callable(name):
            self.activation = name(**params)
        else:
            raise ValueError('Activation should be callable/sigmoid/softmax/logsoftmax/None; got {}'.format(name))

    def forward(self, x):
        return self.activation(x)

In [ ]:
# === IMPORTS ===
# Import necessary libraries for model loading, dataset handling, and visualization
import torch


def _take_channels(*xs, ignore_channels=None):
    if ignore_channels is None:
        return xs
    else:
        channels = [channel for channel in range(xs[0].shape[1]) if channel not in ignore_channels]
        xs = [torch.index_select(x, dim=1, index=torch.tensor(channels).to(x.device)) for x in xs]
        return xs


def _threshold(x, threshold=None):
    if threshold is not None:
        return (x > threshold).type(x.dtype)
    else:
        return x


def iou(pr, gt, eps=1e-7, threshold=None, ignore_channels=None):
    """Calculate Intersection over Union between ground truth and prediction
    Args:
        pr (torch.Tensor): predicted tensor
        gt (torch.Tensor):  ground truth tensor
        eps (float): epsilon to avoid zero division
        threshold: threshold for outputs binarization
    Returns:
        float: IoU (Jaccard) score
    """

    pr = _threshold(pr, threshold=threshold)
    pr, gt = _take_channels(pr, gt, ignore_channels=ignore_channels)

    intersection = torch.sum(gt * pr)
    union = torch.sum(gt) + torch.sum(pr) - intersection + eps
    return (intersection + eps) / union


jaccard = iou


def f_score(pr, gt, beta=1, eps=1e-7, threshold=None, ignore_channels=None):
    """Calculate F-score between ground truth and prediction
    Args:
        pr (torch.Tensor): predicted tensor
        gt (torch.Tensor):  ground truth tensor
        beta (float): positive constant
        eps (float): epsilon to avoid zero division
        threshold: threshold for outputs binarization
    Returns:
        float: F score
    """

    pr = _threshold(pr, threshold=threshold)
    pr, gt = _take_channels(pr, gt, ignore_channels=ignore_channels)

    tp = torch.sum(gt * pr)
    fp = torch.sum(pr) - tp
    fn = torch.sum(gt) - tp

    score = ((1 + beta ** 2) * tp + eps) / ((1 + beta ** 2) * tp + beta ** 2 * fn + fp + eps)

    return score


def accuracy(pr, gt, threshold=0.5, ignore_channels=None):
    """Calculate accuracy score between ground truth and prediction
    Args:
        pr (torch.Tensor): predicted tensor
        gt (torch.Tensor):  ground truth tensor
        eps (float): epsilon to avoid zero division
        threshold: threshold for outputs binarization
    Returns:
        float: precision score
    """
    pr = _threshold(pr, threshold=threshold)
    pr, gt = _take_channels(pr, gt, ignore_channels=ignore_channels)

    tp = torch.sum(gt == pr, dtype=pr.dtype)
    score = tp / gt.view(-1).shape[0]
    return score


def precision(pr, gt, eps=1e-7, threshold=None, ignore_channels=None):
    """Calculate precision score between ground truth and prediction
    Args:
        pr (torch.Tensor): predicted tensor
        gt (torch.Tensor):  ground truth tensor
        eps (float): epsilon to avoid zero division
        threshold: threshold for outputs binarization
    Returns:
        float: precision score
    """

    pr = _threshold(pr, threshold=threshold)
    pr, gt = _take_channels(pr, gt, ignore_channels=ignore_channels)

    tp = torch.sum(gt * pr)
    fp = torch.sum(pr) - tp

    score = (tp + eps) / (tp + fp + eps)

    return score


def recall(pr, gt, eps=1e-7, threshold=None, ignore_channels=None):
    """Calculate Recall between ground truth and prediction
    Args:
        pr (torch.Tensor): A list of predicted elements
        gt (torch.Tensor):  A list of elements that are to be predicted
        eps (float): epsilon to avoid zero division
        threshold: threshold for outputs binarization
    Returns:
        float: recall score
    """

    pr = _threshold(pr, threshold=threshold)
    pr, gt = _take_channels(pr, gt, ignore_channels=ignore_channels)

    tp = torch.sum(gt * pr)
    fn = torch.sum(gt) - tp

    score = (tp + eps) / (tp + fn + eps)

    return score

In [ ]:
# === IMPORTS ===
# Import necessary libraries for model loading, dataset handling, and visualization
import torch.nn as nn

class JaccardLoss(Loss):
    def __init__(self, eps=1.0, activation=None, ignore_channels=None, **kwargs):
        super().__init__(**kwargs)
        self.eps = eps
        self.activation = Activation(activation)
        self.ignore_channels = ignore_channels

    def forward(self, y_pr, y_gt):
        y_pr = self.activation(y_pr)
        return 1 - jaccard(
            y_pr,
            y_gt,
            eps=self.eps,
            threshold=None,
            ignore_channels=self.ignore_channels,
        )


class DiceLoss(Loss):
    def __init__(self, eps=1.0, beta=1.0, activation=None, ignore_channels=None, **kwargs):
        super().__init__(**kwargs)
        self.eps = eps
        self.beta = beta
        self.activation = Activation(activation)
        self.ignore_channels = ignore_channels

    def forward(self, y_pr, y_gt):
        y_pr = self.activation(y_pr)
        return 1 - f_score(
            y_pr,
            y_gt,
            beta=self.beta,
            eps=self.eps,
            threshold=None,
            ignore_channels=self.ignore_channels,
        )


class L1Loss(nn.L1Loss, Loss):
    pass


class MSELoss(nn.MSELoss, Loss):
    pass


class CrossEntropyLoss(nn.CrossEntropyLoss, Loss):
    pass


class NLLLoss(nn.NLLLoss, Loss):
    pass


class BCELoss(nn.BCELoss, Loss):
    pass


class BCEWithLogitsLoss(nn.BCEWithLogitsLoss, Loss):
    pass

    # https://www.scitepress.org/Papers/2021/103040/103040.pdf

In [ ]:
# === EVALUATION ===
# Compute IoU, Dice coefficient, or other metrics for quantitative evaluation
class IoU(Metric):
    __name__ = "iou_score"

    def __init__(self, eps=1e-7, threshold=0.5, activation=None, ignore_channels=None, **kwargs):
        super().__init__(**kwargs)
        self.eps = eps
        self.threshold = threshold
        self.activation = Activation(activation)
        self.ignore_channels = ignore_channels

    def forward(self, y_pr, y_gt):
        y_pr = self.activation(y_pr)
        return iou(
            y_pr,
            y_gt,
            eps=self.eps,
            threshold=self.threshold,
            ignore_channels=self.ignore_channels,
        )


class Fscore(Metric):
    def __init__(self, beta=1, eps=1e-7, threshold=0.5, activation=None, ignore_channels=None, **kwargs):
        super().__init__(**kwargs)
        self.eps = eps
        self.beta = beta
        self.threshold = threshold
        self.activation = Activation(activation)
        self.ignore_channels = ignore_channels

    def forward(self, y_pr, y_gt):
        y_pr = self.activation(y_pr)
        return f_score(
            y_pr,
            y_gt,
            eps=self.eps,
            beta=self.beta,
            threshold=self.threshold,
            ignore_channels=self.ignore_channels,
        )


class Accuracy(Metric):
    def __init__(self, threshold=0.5, activation=None, ignore_channels=None, **kwargs):
        super().__init__(**kwargs)
        self.threshold = threshold
        self.activation = Activation(activation)
        self.ignore_channels = ignore_channels

    def forward(self, y_pr, y_gt):
        y_pr = self.activation(y_pr)
        return accuracy(
            y_pr,
            y_gt,
            threshold=self.threshold,
            ignore_channels=self.ignore_channels,
        )


class Recall(Metric):
    def __init__(self, eps=1e-7, threshold=0.5, activation=None, ignore_channels=None, **kwargs):
        super().__init__(**kwargs)
        self.eps = eps
        self.threshold = threshold
        self.activation = Activation(activation)
        self.ignore_channels = ignore_channels

    def forward(self, y_pr, y_gt):
        y_pr = self.activation(y_pr)
        return recall(
            y_pr,
            y_gt,
            eps=self.eps,
            threshold=self.threshold,
            ignore_channels=self.ignore_channels,
        )


class Precision(Metric):
    def __init__(self, eps=1e-7, threshold=0.5, activation=None, ignore_channels=None, **kwargs):
        super().__init__(**kwargs)
        self.eps = eps
        self.threshold = threshold
        self.activation = Activation(activation)
        self.ignore_channels = ignore_channels

    def forward(self, y_pr, y_gt):
        y_pr = self.activation(y_pr)
        return precision(
            y_pr,
            y_gt,
            eps=self.eps,
            threshold=self.threshold,
            ignore_channels=self.ignore_channels,
        )

In [ ]:
metrics = [
    IoU(threshold=0.5),
    Accuracy(threshold=0.5),
    Fscore(threshold=0.5),
    Recall(threshold=0.5),
    Precision(threshold=0.5),
]

In [ ]:
# === MODEL LOADING ===
# Load the pre-trained binary segmentation model and set to evaluation mode
# install and Clone Github segmentation models pytorch
!pip install git+https://github.com/qubvel/segmentation_models.pytorch

  Cloning https://github.com/qubvel/segmentation_models.pytorch to /tmp/pip-req-build-1kz44wvx
  Running command git clone --filter=blob:none --quiet https://github.com/qubvel/segmentation_models.pytorch /tmp/pip-req-build-1kz44wvx
  Resolved https://github.com/qubvel/segmentation_models.pytorch to commit 6db76a1106426ac5b55f39fba68168f3bccae7f8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
# === IMPORTS ===
# Import necessary libraries for model loading, dataset handling, and visualization
import torch
import segmentation_models_pytorch as smp

In [ ]:
# === DATA AUGMENTATION AND PREPROCESSING ===
# Define augmentation and preprocessing pipelines for validation and inference
# resnext50_32x4d, mit_b2, timm-gernet_s, efficientnet-b3, mobilenet_v2, resnet152, vgg13		# EXPERIMENT HERE WITH TRUE DATA
ENCODER = 'efficientnet-b3' # extracts feature map and increases channels
ENCODER_WEIGHTS = 'imagenet'
ACTIVATION = 'sigmoid'

# create segmentation model with pretrained encoder
# decoders= PAN, PSPNet, MAnet, Linknet, FPN, DeepLabV3, DeepLabV3Plus, Unet  # upsamples feature maps and decreases channels and outputs segmentation maps    # EXPERIMENT HERE WITH TRUE DATA
model =smp.FPN(
    encoder_name=ENCODER,
    encoder_weights=ENCODER_WEIGHTS,
    classes=len(CLASSES),
    activation=ACTIVATION,
)
# normalize the data the same way as during encoder weight pre-training
preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)


# smp is the library for encoders and decoders
# resnet encoder commonly used in image segmentation
# pspnet architecture: can use any encoder, used for semantic segmentation, cnn/pyramid pooling
# pan architecture (pyramid attention network): good for semantic segmentation
# unet arhcitecture: semantic segmentation for biomedical images
# do: resnet50/unet or ++, efficientnet-b3/fpn, mobilenet_v2/pspnet

In [ ]:
# === MODEL LOADING ===
# Load the pre-trained binary segmentation model and set to evaluation mode
# define optimization algorithm with learning rate # EXPERIMENT WITH RMSPROP AND ADAM
optimizer = torch.optim.Adam([
    dict(params=model.parameters(), lr=0.001), #0.0001 # EXPERIMENT HERE WITH TRUE DATA
])

# define loss function
loss = BCEWithLogitsLoss() #DiceLoss() # EXPERIMENT HERE WITH BOTH LOSSES

In [ ]:
# === IMPORTS ===
# Import necessary libraries for model loading, dataset handling, and visualization
from torch.utils.data import DataLoader

In [ ]:
# === IMPORTS ===
# Import necessary libraries for model loading, dataset handling, and visualization
import numpy as np


class Meter(object):
    """Meters provide a way to keep track of important statistics in an online manner.
    This class is abstract, but provides a standard interface for all meters to follow.
    """

    def reset(self):
        """Reset the meter to default settings."""
        pass

    def add(self, value):
        """Log a new value to the meter
        Args:
            value: Next result to include.
        """
        pass

    def value(self):
        """Get the value of the meter in the current state."""
        pass


class AverageValueMeter(Meter):
    def __init__(self):
        super(AverageValueMeter, self).__init__()
        self.reset()
        self.val = 0

    def add(self, value, n=1):
        self.val = value
        self.sum += value
        self.var += value * value
        self.n += n

        if self.n == 0:
            self.mean, self.std = np.nan, np.nan
        elif self.n == 1:
            self.mean = 0.0 + self.sum  # this is to force a copy in torch/numpy
            self.std = np.inf
            self.mean_old = self.mean
            self.m_s = 0.0
        else:
            self.mean = self.mean_old + (value - n * self.mean_old) / float(self.n)
            self.m_s += (value - self.mean_old) * (value - self.mean)
            self.mean_old = self.mean
            self.std = np.sqrt(self.m_s / (self.n - 1.0))

    def value(self):
        return self.mean, self.std

    def reset(self):
        self.n = 0
        self.sum = 0.0
        self.var = 0.0
        self.val = 0.0
        self.mean = np.nan
        self.mean_old = 0.0
        self.m_s = 0.0
        self.std = np.nan

In [ ]:
# === IMPORTS ===
# Import necessary libraries for model loading, dataset handling, and visualization
import sys
import torch
from tqdm import tqdm as tqdm


In [ ]:
# === INFERENCE ===
# Run inference on test images and generate segmentation predictions
class Epoch:
    def __init__(self, model, loss, metrics, stage_name, device="cpu", verbose=True):
        self.model = model
        self.loss = loss
        self.metrics = metrics
        self.stage_name = stage_name
        self.verbose = verbose
        self.device = device

        self._to_device()

    def _to_device(self):
        self.model.to(self.device)
        if self.loss is not None:
            self.loss.to(self.device)
        for metric in self.metrics:
            metric.to(self.device)

    def _format_logs(self, logs):
        str_logs = ["{} - {:.4}".format(k, v) for k, v in logs.items()]
        s = ", ".join(str_logs)
        return s

    def batch_update(self, x, y):
        raise NotImplementedError

    def on_epoch_start(self):
        pass

    def run(self, dataloader, is_test=False):
        self.on_epoch_start()

        if is_test:
            return self._run_test(dataloader)

        logs = {}
        loss_meter = AverageValueMeter()
        metrics_meters = {metric.__name__: AverageValueMeter() for metric in self.metrics}

        with tqdm(
            dataloader,
            desc=self.stage_name,
            file=sys.stdout,
            disable=not (self.verbose),
        ) as iterator:
            for x, y in iterator:
                x, y = x.to(self.device), y.to(self.device)
                loss, y_pred = self.batch_update(x, y)

                # update loss logs
                loss_value = loss.cpu().detach().numpy()
                loss_meter.add(loss_value)
                loss_logs = {self.loss.__name__: loss_meter.mean}
                logs.update(loss_logs)

                # update metrics logs
                for metric_fn in self.metrics:
                    metric_value = metric_fn(y_pred, y).cpu().detach().numpy()
                    metrics_meters[metric_fn.__name__].add(metric_value)
                metrics_logs = {k: v.mean for k, v in metrics_meters.items()}
                logs.update(metrics_logs)

                if self.verbose:
                    s = self._format_logs(logs)
                    iterator.set_postfix_str(s)

        return logs

    def _run_test(self, dataloader):
        predictions = []
        with tqdm(
            dataloader,
            desc="test",
            file=sys.stdout,
            disable=not self.verbose
        ) as iterator:
            for batch in iterator:
                x = batch.to(self.device)
                # print("Batch shape in _run_test:", x.shape)  # Debugging line
                y_pred = self.batch_update(x)
                print("Number of predictions in this batch:", y_pred.size(0))  # assuming y_pred is of shape [batch_size, ...]
                predictions.extend(y_pred.cpu())  # use extend if y_pred contains multiple images
                # predictions.append(y_pred.cpu())

        return predictions

In [ ]:
# === MODEL LOADING ===
# Load the pre-trained binary segmentation model and set to evaluation mode
class ValidEpoch(Epoch):
    def __init__(self, model, loss, metrics, device="cpu", verbose=True):
        super().__init__(
            model=model,
            loss=loss,
            metrics=metrics,
            stage_name="valid",
            device=device,
            verbose=verbose,
        )

    def on_epoch_start(self):
        self.model.eval()

    # def batch_update(self, x, y=None):
    #     with torch.no_grad():
    #         prediction = self.model.forward(x)
    #         return prediction

    def batch_update(self, x, y=None):
        with torch.no_grad():
            print("Input shape in batch_update:", x.shape)  # Debugging line
            prediction = self.model.forward(x)
            return prediction

In [ ]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# === IMPORTS ===
# Import necessary libraries for model loading, dataset handling, and visualization
# validation transformations are used for test data because they are less aggressive and maintain the original content of the image, which is important for evaluation

In [ ]:
# === DATASET AND DATALOADER ===
# Define or load dataset class and data loader for testing and evaluation


class TestDataset(BaseDataset):
    """Dataset for testing. Read images, apply preprocessing transformations."""

    def __init__(self, images_dir, augmentation=None, preprocessing=None):
        self.ids = sorted(os.listdir(images_dir))
        self.images_fps = [os.path.join(images_dir, image_id) for image_id in self.ids]
        self.augmentation = augmentation
        self.preprocessing = preprocessing

    def __getitem__(self, i):
        # read data
        image = cv2.imread(self.images_fps[i])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # apply augmentations
        if self.augmentation:
            sample = self.augmentation(image=image)
            image = sample['image']

        # apply preprocessing
        if self.preprocessing:
            sample = self.preprocessing(image=image)
            image = sample['image']

        return image

    def __len__(self):
        return len(self.ids)

In [ ]:
# def get_test_augmentation():
#     """Add paddings to make image shape divisible by 32"""
#     test_transform = [
#         albu.PadIfNeeded(544, 960)
#     ]
#     return albu.Compose(test_transform)

In [ ]:
# === DATASET AND DATALOADER ===
# Define or load dataset class and data loader for testing and evaluation


test_dataset = TestDataset(
    images_dir='/content/drive/My Drive/Colab Notebooks/micc-278/real_images_test/',
    augmentation=get_validation_augmentation(),
    preprocessing=get_preprocessing(preprocessing_fn)
)

batch_size = 4  # adjust this based on system's capabilities
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
for batch in test_dataloader:
    x = batch
    print("Batch shape:", x.shape)  # Should print [batch_size, C, H, W]
    break

Batch shape: torch.Size([4, 3, 544, 960])


In [ ]:
print("Total number of images in dataset:", len(test_dataset))

In [ ]:
# === IMPORTS ===
# Import necessary libraries for model loading, dataset handling, and visualization
import os

save_dir = '/content/drive/My Drive/Colab Notebooks/micc-278/PredictedImages'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [ ]:
# === MODEL LOADING ===
# Load the pre-trained binary segmentation model and set to evaluation mode
Trained_model = torch.load('/content/drive/My Drive/Colab Notebooks/challenge_data_trained_binary_model.pth')
test_epoch = ValidEpoch(
    model=Trained_model,
    loss=None,  # Omit loss if predicting without calculating loss
    metrics=metrics,  # Assuming metrics is defined, otherwise set it to []
    device=DEVICE,
)

predictions = test_epoch.run(test_dataloader, is_test=True)


test:   0%|          | 0/12 [00:00<?, ?it/s]Input shape in batch_update: torch.Size([4, 3, 544, 960])
Number of predictions in this batch: 4
test:   8%|▊         | 1/12 [00:00<00:03,  2.85it/s]Input shape in batch_update: torch.Size([4, 3, 544, 960])
Number of predictions in this batch: 4
test:  17%|█▋        | 2/12 [00:00<00:03,  3.00it/s]Input shape in batch_update: torch.Size([4, 3, 544, 960])
Number of predictions in this batch: 4
test:  25%|██▌       | 3/12 [00:00<00:02,  3.08it/s]Input shape in batch_update: torch.Size([4, 3, 544, 960])
Number of predictions in this batch: 4
test:  33%|███▎      | 4/12 [00:01<00:02,  3.15it/s]Input shape in batch_update: torch.Size([4, 3, 544, 960])
Number of predictions in this batch: 4
test:  42%|████▏     | 5/12 [00:01<00:02,  3.21it/s]Input shape in batch_update: torch.Size([4, 3, 544, 960])
Number of predictions in this batch: 4
test:  50%|█████     | 6/12 [00:01<00:01,  3.24it/s]Input shape in batch_update: torch.Size([4, 3, 544, 960])
Num

In [ ]:
# test_dataset = cv.copyMakeBorder(test_dataset,0,0,2,2,cv.BORDER_REPLICATE)
# https://docs.opencv.org/4.x/d3/df2/tutorial_py_basic_ops.html

In [ ]:
# === IMPORTS ===
# Import necessary libraries for model loading, dataset handling, and visualization
import torch
from torchvision.utils import save_image

for i, prediction in enumerate(predictions):
    # invert colors: change instruments to white and background to black
    inverted_prediction = 1 - prediction

    # ensure the tensor values are still in the range [0, 1]
    inverted_prediction = torch.clamp(inverted_prediction, 0, 1)

    save_path = os.path.join(save_dir, f'prediction_{i}.png')
    save_image(inverted_prediction, save_path)

In [ ]:
# === INFERENCE ===
# Run inference on test images and generate segmentation predictions
for i, prediction in enumerate(predictions):
    # invert colors: change instruments to white and background to black
    inverted_prediction = 1 - prediction

    # ensure the tensor values are still in the range [0, 1]
    inverted_prediction = torch.clamp(inverted_prediction, 0, 1)

    save_path = os.path.join(save_dir, f'prediction_{i}.png')
    save_image(inverted_prediction, save_path)

In [ ]:
# === IMPORTS ===
# Import necessary libraries for model loading, dataset handling, and visualization
# import os
# from torchvision.utils import save_image

# for i, prediction in enumerate(predictions):
#     save_path = os.path.join(save_dir, f'prediction_{i}.png')
#     save_image(prediction, save_path)

In [ ]:
# === INFERENCE ===
# Run inference on test images and generate segmentation predictions
# IoU = intersection over union (overlap)
# y_pred = model.predict(an_x_test_item)
# y_pred_thresholded = y_pred > 0.5

In [ ]:
# === IMPORTS ===
# Import necessary libraries for model loading, dataset handling, and visualization
# from tensorflow.keras.metrics import MeanIoU

In [ ]:
# === INFERENCE ===
# Run inference on test images and generate segmentation predictions
# n_classes = 2
# IOU_keras = MeanIoU(num_classes=n_classes)
# IOU_keras.update_state(y_pred_thresholded, y_test)
# print("Mean IoU =", IOU_keras.result().numpy())
# 1.0 avg is a red flag....maybe the 0 and 1 for target are switched and it's predicting for black????

In [ ]:
# === INFERENCE ===
# Run inference on test images and generate segmentation predictions
# threshold = 0.5
# test_img_number = random.randint(0, len(X_test)-1)
# test_img = X_test[test_img_number]
# ground_truth=y_test[test_img_number]
# test_img_input=np.expand_dims(test_img, 0)
# print(test_img_input.shape)
# prediction = (model.predict(test_img_input)[0,:,:,0] > 0.5).astype(np.uint8)
# print(prediction.shape)

# plt.figure(figsize=(16, 8))
# plt.subplot(231)
# plt.title('Testing Image')
# plt.imshow(test_img[:,:,0], cmap='gray')
# plt.subplot(232)
# plt.title('Testing Label')
# plt.imshow(ground_truth[:,:,0], cmap='gray')
# plt.subplot(233)
# plt.title('Prediction on test image')
# plt.imshow(prediction, cmap='gray')

# plt.show()

In [ ]:
# # plot the training and validation accuracy and loss at each epoch
# loss = Trained_model.history['loss']
# val_loss = Trained_model.history['val_loss']
# epochs = range(1, len(loss) + 1)
# plt.plot(epochs, loss, 'y', label='Training loss')
# plt.plot(epochs, val_loss, 'r', label='Validation loss')
# plt.title('Training and validation loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()
# plt.show()

# acc = Trained_model.history['accuracy']
# val_acc = Trained_model.history['val_accuracy']
# plt.plot(epochs, acc, 'y', label='Training acc')
# plt.plot(epochs, val_acc, 'r', label='Validation acc')
# plt.title('Training and validation accuracy')
# plt.xlabel('Epochs')
# plt.ylabel('Accuracy')
# plt.legend()
# plt.show()

In [ ]:
# === IMPORTS ===
# Import necessary libraries for model loading, dataset handling, and visualization
import pandas as pd

In [ ]:
results_dict = {'Metric Name': [], 'Value': []}

In [ ]:
for metric_name, metric_value in logs.items():
    results_dict['Metric Name'].append(metric_name)
    results_dict['Value'].append(metric_value)

In [ ]:
results_df = pd.DataFrame(results_dict)
print(results_df)

            Metric Name     Value
0  bce_with_logits_loss  0.374519
1             iou_score  0.997403
2              accuracy  0.997863
3                fscore  0.998700
4                recall  0.998260
5             precision  0.999140


In [ ]:
results_df.to_csv('/content/drive/My Drive/Colab Notebooks/MICC-284-results.csv', index=False)

In [ ]:
# === MODEL LOADING ===
# Load the pre-trained binary segmentation model and set to evaluation mode
for i in range(len(test_dataset)):
  image, gt_mask = test_dataset[i]

  x_tensor = torch.from_numpy(image).to(DEVICE).unsqueeze(0)
  predicted_mask = Trained_model.predict(x_tensor)

  pr_mask = (predicted_mask.squeeze().cpu().numpy().round())
  pr_mask = pr_mask[:,:]    #[1,:,:]
  gt_mask = gt_mask[0,:]    #[1,:,:]
  image_t = image.transpose(1, 2, 0)
  visualizeData(
    image=image_t,
    ground_truth_mask=gt_mask,
    predicted_mask=pr_mask
  )

In [ ]:
# === MODEL LOADING ===
# Load the pre-trained binary segmentation model and set to evaluation mode
# # get orignial image and mask from test dataset
# image, gt_mask = test_dataset[9]  # change the index number to try out diff test images

# x_tensor = torch.from_numpy(image).to(DEVICE).unsqueeze(0)
# predicted_mask = Trained_model.predict(x_tensor)

# pr_mask = (predicted_mask.squeeze().cpu().numpy().round())
# pr_mask = pr_mask[:,:]    #[1,:,:]
# gt_mask = gt_mask[0,:]    #[1,:,:]
# image_t = image.transpose(1, 2, 0)
# visualizeData(
#         image=image_t,
#         ground_truth_mask=gt_mask,
#         predicted_mask=pr_mask
#     )
# This error occurs when I try to access an element in a NumPy array using too many indices. The error message specifically means that I am trying to index a 2-dimensional NumPy array with 3 indices.

In [ ]:
# === INFERENCE ===
# Run inference on test images and generate segmentation predictions
# convert the predicted mask to numpy and get the predicted class indices
# predicted_output = torch.argmax(predicted_mask.squeeze(), dim=0).detach().cpu().numpy()
# Indices = np.unique(predicted_output)

# # print(Indices)
# print(CLASSES)

['instrument']
